# Chlorophyll Project
#### There are seventy seabird observational sites across the globe. We want to gather chlorophyll data for a 1km diameter around these points, in a circle.

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
import matplotlib.cbook as cbook

#load map packages
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from PIL import Image

In [2]:
# make a function (called below) that makes a mask (2d dataset where values to retain are set to 1 and others to nan)
# for a 300km radius area around a given lat/lon pair. 
# note that the area in km is not the same as area in degrees. longitude degrees change distance with latitude.

In [3]:
# if acquiring for the first time (1)
ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v5.0-MONTHLY')

In [4]:
# open CSV
locs = pd.read_csv('../NSF_Seabirdsloc_Summer2022.csv')
sites = locs['Site']

# for now, let's take an input. later it will be a function
site = input("Input Desired Site")
if (site == ""):
    site = 'Adams'
sites = sites.tolist()

lon = locs.loc[[sites.index(site)]]['Longitude']
lat = locs.loc[[sites.index(site)]]['Latitude']

print("Working site " + site)

Working site Adams


In [5]:
# simple deg2rad
def radians(degrees):
    return degrees * Math.pi / 180

# gets distance between two points in longitude and latitude space
def getDistance(lat1, lon1, lat2, lon2):
    R = 6371 # Radius of the earth in km
    dLat = radians(lat2-lat1)
    dLon = radians(lon2-lon1)
    a = np.sin(dLat/2) * np.sin(dLat/2) + np.cos(radians(lat1)) * np.cos(radians(lat2)) * np.sin(dLon/2) * np.sin(dLon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c # Distance in km
    return d

In [14]:
# open CSV
# for each location, select a region +-5 deg, all times
dist = 2.702; # 300 km range
lats = [float(lat-dist), float(lat+dist)]
lons = [float(lon-dist), float(lon+dist)]

data = ds_color['chlor_a'][291].sel(
    lon = slice(lons[0], lons[1])
).load()

# Select a region of our data
fig = plt.figure(figsize=(16,10), dpi = 72)
ax = plt.axes(projection=ccrs.PlateCarree())


region = np.array([[lats[0],lats[1]],[lons[0],lons[1]]]) 

states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m')

ax.coastlines(resolution='10m',linewidth=1,color='black')
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(states_provinces, linewidth = 0.5)
ax.add_feature(cfeature.BORDERS, color = 'black')
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.gridlines(linestyle = '--', linewidth = 0.5)

#plot diff data
plt.title("Seabird Observation Locations")

#print lats and lons range
print("Mapping between " + str(lats[0]) + " and " + str(lats[1]))
print("Mapping between " + str(lons[0]) + " and " + str(lons[1]))


data.plot(cmap="viridis")


# plot point

# UNCOMMENT for location labels
# for x in range( len( locs["Longitude"] ) ): 
#     plt.text(
#         locs["Longitude"][x], locs["Latitude"][x],
#         locs["Site"][x] + ", " + locs["Location"][x],
#         ha='center')

# call mask function 

# pick a random point in the data, and calculate its distance to the site.

# take a mean in space, and store the time series into a numpy array

# save the arrays as dataarray in a dataset

# add locations as other variables 

#get path
path = os.getcwd() + "\\"

plt.savefig(path + 'graph.png', dpi=72, bbox_inches='tight', pad_inches=0)

background = Image.open(path + "graph.png").convert("RGBA")
base = Image.new("RGBA", background.size, "WHITE")
foreground = Image.open(path + "assets/overlay.png").convert("RGBA")

base.paste(background, (0, 0), background)
base.paste(foreground, (0, 0), foreground)
base.save("../output/graph.png")
# base.show()
# display(background)
print("Output saved to folder as graph.png")

Mapping between -53.6047 and -48.200700000000005
Mapping between 163.2998 and 168.7038
Output saved to folder as graph.png


In [7]:
# save the file into a netcdf
